In [21]:
import numpy as np
import pickle
import os
import sys
import importlib
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [22]:
util_path = 'C:/ASM/Dropbox/Developments/Jupyter/Eating/myutils' if 'C:' in os.getcwd() else './myutils'
sys.path.append(util_path)
import my_file_utils as mfileu
import my_classification_utils as mclfu
import my_steven_lab_utils as mslabu
#importlib.reload(biteu)

In [37]:
win_size, axis_count = 5*16, 6
print("Win size:", win_size, ", axis count: ", axis_count)

Win size: 80 , axis count:  6


In [24]:
def normalize_data(ds):
    new_ds= []    
    for subj in range(len(ds)):
        subj_ds=[]
        for sess in range(len(ds[subj])):
            d = ds[subj][sess]
            accel = d[:, 1:4]
            accel = (accel+3*9.8)/(2*3*9.8)
            accel[accel>1] = 1
            accel[accel<1] = 0
            
            gyro = d[:, 4:7]
            gyro = (gyro+10)/(2*10)
            gyro[gyro>1] = 1
            gyro[gyro<0] = 0
            
            t = d[:, 0].reshape((-1, 1))
            d = np.concatenate((t, accel, gyro), axis=1)
            subj_ds.append(d)
            
            assert np.sum(d[:, 1:]>1) == 0
            assert np.sum(d[:, 1:]<0) == 0
            assert d.shape[1] == 7
            
        new_ds.append(subj_ds)
        
    return new_ds            

In [25]:
def get_window_data(ds, indices):
    count = len(indices)
    w = np.zeros((count, win_size, 6))        
    
    for i in range(count):
        subj, sess, ix = indices[i, 0], indices[i, 1], indices[i, 2]
        w[i, :, :] = ds[subj][sess][ix:ix+win_size, 1:7]                
        
    return w

In [26]:
subj, num_epochs, train_test = 0, 1, 'train'
if 'C:' not in mfileu.get_path():    
    subj, num_epochs, train_test = int(sys.argv[1]), int(sys.argv[2]), sys.argv[3]
    
assert train_test in ['train', 'test_lab', 'test_free']

In [27]:
FILTER_SIZE = 5
NUM_FILTERS = 64
BATCH_SIZE = 100
NUM_SENSOR_CHANNELS = 6
NUM_UNITS_LSTM = 128

In [28]:
def conv_layer(x, size_in, size_out, ksize, strides, padding, name):    
    strides = [1, strides[0], strides[1], 1]   
    with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal([ksize[0], ksize[1], size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.0, shape=[size_out]), name="b")
        conv = tf.nn.conv2d(x, W, strides=strides, padding=padding)
        output = tf.nn.relu(conv + b)
        
    return output

In [29]:
def lstm_layer(lstm_input, name):
    cell_fw = tf.nn.rnn_cell.LSTMCell(NUM_UNITS_LSTM, state_is_tuple=True)        
    initial_state = cell_fw.zero_state(BATCH_SIZE, tf.float32)

    output, last_state = tf.nn.dynamic_rnn(cell_fw, lstm_input,
                                           initial_state=initial_state,                                               
                                           scope = name,
                                           dtype=tf.float32)
    return output

In [30]:
def fc_layer(x, size_out, name="FC_Layer" ):
    dims = x.get_shape().as_list()
    size_in = dims[-1]
    with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.0, shape=[size_out]), name="b")
        output = tf.matmul(x, W) + b             
        
    return output   

In [65]:
def get_net(x):
    x_shape =x.get_shape().as_list()
    print('Inside get_net,  x_shape: ', x_shape)
    
    x = tf.reshape(x, shape=[-1, x.shape[1], x.shape[2], 1], name="reshape")
    conv_1 = conv_layer(x, size_in=1, size_out=64, ksize=[5,1], strides=[1,1], padding="VALID", name='conv_1')                
    conv_2 = conv_layer(conv_1, size_in=64, size_out=64, ksize=[5,1], strides=[1,1], padding="VALID", name='conv_2')
    conv_3 = conv_layer(conv_2, size_in=64, size_out=64, ksize=[5,1], strides=[1,1], padding="VALID", name='conv_3')
    conv_4 = conv_layer(conv_3, size_in=64, size_out=64, ksize=[5,1], strides=[1,1], padding="VALID", name='conv_4')

    print("Conv Layer Shapes: ")
    print("Conv Layer 1: ", conv_1.get_shape().as_list())
    print("Conv Layer 2: ", conv_2.get_shape().as_list())
    print("Conv Layer 3: ", conv_3.get_shape().as_list())
    print("Conv Layer 4: ", conv_4.get_shape().as_list())
    
    sz = conv_4.get_shape().as_list()
    lstm_input = tf.reshape(conv_4, shape=[-1, sz[1], sz[2]*sz[3]], name="Flattened")
    
    lstm_1 = lstm_layer(lstm_input, name="lstm_1")    
    lstm_2 = lstm_layer(lstm_1, name="lstm_2")        
    
    lstm_out = lstm_2[:, -1, :]    
    print("Lstm out shape: ", lstm_out.get_shape().as_list())
    
    logits = fc_layer(lstm_out, 1, name="Logits")  
    print("Logit shape: ",logits.get_shape().as_list())
    
    return logits    

In [66]:
def train_test_model(ds, train_indices, test_indices, model_path_dest=None, model_path_src=None):       
    #print_out, sys.stdout = sys.stdout, open(os.devnull, 'w')    
    
    tf.reset_default_graph()
    
    x = tf.placeholder(tf.float32, [None, win_size, axis_count], name="x")        
    y = tf.placeholder(tf.float32, [None, 1], name="y")    
    
    logits = get_net(x)
    prediction = tf.nn.sigmoid(logits, name="prediction")
    correct_prediction = tf.equal(tf.greater(prediction, 0.5), tf.equal(y,1), name="correct_prediction")
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    
    loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y), name="loss_op")        
    train_step = tf.train.AdamOptimizer().minimize(loss_op, name="train_step")

    sess = tf.Session()
    
    #sys.stdout = print_out
    ########## Train and then save the model ########################
    if len(train_indices)>0:                
        sess.run(tf.global_variables_initializer())    
        
        train_indices, _ = mclfu.adjust_for_batch_size(train_indices, train_indices, BATCH_SIZE)
        train_count = len(train_indices)
        for epoch in range(num_epochs):
            print("Epoch:", epoch)
            total_loss, total_acc = 0, 0
            for ix in range(0, train_count, BATCH_SIZE):                                            
                batch_x = get_window_data(ds, train_indices[ix:ix+BATCH_SIZE])                 
                batch_y = train_indices[ix:ix+BATCH_SIZE, -1].reshape((-1,1))                 
                sess.run(train_step, feed_dict={x:batch_x, y:batch_y})                

                loss, acc = sess.run([loss_op, accuracy], feed_dict={x:batch_x, y:batch_y})        
                total_loss+= loss*BATCH_SIZE
                total_acc += acc*BATCH_SIZE                
            print('  Train loss: {:.4f}, acc: {:.4f}'.format(total_loss/train_count, total_acc/train_count))

            test_count = len(test_indices)
            if test_count>0:            
                test_indices, _ = mclfu.adjust_for_batch_size(test_indices, test_indices, BATCH_SIZE)
                total_loss, total_acc = 0, 0
                for ix in range(0, test_count, BATCH_SIZE):                
                    batch_x, batch_features = get_window_data(ds, test_indices[ix:ix+BATCH_SIZE])
                    batch_y = test_indices[ix:ix+BATCH_SIZE, -1].reshape((-1,1))                      
                    loss, acc = sess.run([loss_op, accuracy], feed_dict={x:batch_x, y:batch_y})                
                    total_loss+= loss*BATCH_SIZE
                    total_acc += acc*BATCH_SIZE                
                print('  Test loss: {:.4f}, acc: {:.4f}'.format(total_loss/test_count, total_acc/test_count))

        print('!!!!!!!!!!!!!!! Optimization Finished !!!!!!!!!!!!!!!!!')

        if model_path_dest:
            saver = tf.train.Saver()            
            mfileu.create_directory(model_path_dest)
            saver.save(sess, model_path_dest+'/model')    
            print("Model Saved!")
        sess.close()
        
    ########## Restore the model and then Test  ########################
    else:
        saver = tf.train.Saver()
        saver.restore(sess, model_path_src+'/model')
        print("Model Loaded for test!")
        
        test_count_original = len(test_indices)        
        test_indices, _ = mclfu.adjust_for_batch_size(test_indices, test_indices, BATCH_SIZE)
        test_count = len(test_indices)
        res = np.zeros((test_count, 1))
        
        for ix in range(0, test_count, BATCH_SIZE):                
            batch_x, batch_features = get_window_data(ds, test_indices[ix:ix+BATCH_SIZE], win_size)
            batch_y = test_indices[ix:ix+BATCH_SIZE, -1].reshape((-1,1))  
            pred = sess.run([prediction], feed_dict={x: batch_x, y:batch_y})            
            res[ix:ix+BATCH_SIZE, 0] = np.array(pred).reshape((-1, ))
        
        res = res[:test_count_original, :]        
        sess.close()
        return res
        

In [67]:
print("==================== Subject, Epochs, Win Size: {}, {}, {} =============".format(subj, num_epochs, win_size))
print("=========================== Train/Test: {} =============================".format(train_test))

==================== Subject, Epochs, Win Size: 0, 1, 80 =============
=========================== Train/Test: train =============================


In [68]:
path = mfileu.get_path()
model_folder_src = path+'/bite_models_DeepConvLSTM'
model_folder_dest = model_folder_src

In [69]:
if train_test == 'train':
    print("Training.....")
    ds = mfileu.read_file('data', 'lab_data_steven.pkl')
    ds, _, _ = mslabu.separate_right_left_annots(ds)    
    ds = normalize_data(ds)    
    indices = mfileu.read_file('features', 'lab_labels_steven_single_array.pkl')
    
    indices = indices[indices[:,0]!=subj, :]        
    assert np.sum(indices[:, 0]==subj) == 0    
    assert np.sum(indices[:, -1]>1) == 0
    assert np.sum(indices[:, -1]<0) == 0
    print("Indices summary after subject filter total, neg, pos:", len(indices), np.sum(indices[:, -1]==0), np.sum(indices[:, -1]==1))
        
    indices = shuffle(indices)    
    train_indices, val_indices = train_test_split(indices, test_size=0.1, stratify=indices[:, -1])
    
    print("train, val shapes: ", train_indices.shape, val_indices.shape, np.sum(train_indices[:, -1]), np.sum(val_indices[:, -1]))
    
    model_path_dest = model_folder_dest+"/subj_"+str(subj)
    train_test_model(ds, train_indices=train_indices, test_indices=val_indices, model_path_dest=model_path_dest)  

Training.....
Indices summary after subject filter total, neg, pos: 1914621 1895991 18630
train, val shapes:  (1723158, 4) (191463, 4) 16767 1863
Inside get_net,  x_shape:  [None, 80, 6]
Conv Layer Shapes: 
Conv Layer 1:  [None, 76, 6, 64]
Conv Layer 2:  [None, 72, 6, 64]
Conv Layer 3:  [None, 68, 6, 64]
Conv Layer 4:  [None, 64, 6, 64]
Lstm out shape:  [100, 128]
Logit shape:  [100, 1]
Epoch: 0


KeyboardInterrupt: 

In [ ]:
if train_test=='test_lab':
    print("Bite Model Testing Lab .....")
    ds = mfileu.read_file('data', 'lab_data_steven_smoothed.pkl')
    ds, _, _ = mslabu.separate_right_left_annots(ds)
    ds = normalize_data(ds) 
    indices = mfileu.read_file('features', 'lab_labels_steven_dict.pkl')
        
    res = {}    
    for subj in range(len(ds)):        
        for sess in range(len(ds[subj])):
            test_indices[:, 2] = indices[(subj, sess)]            
            print("Subj, sess, indices shape: ", subj, sess, test_indices.shape)
                
            model_path_src = model_folder_src+"/subj_"+str(subj)            
            pred = train_test_model(ds, train_indices=[], test_indices=test_indices, model_path_src=model_path_src)            
            print("Prediction shape: ", pred.shape, np.sum(pred), np.sum(pred>=0.5))            
            assert len(pred)==len(test_indices)
            print("Sample Predictions :", pred[:20])
            
            ixpred = np.zeros((len(pred), 2))
            ixpred[:, 0] = ix
            ixpred[:, 1] = pred            
            res[(subj, sess)] = ixpred
            print("Result shape, total_prob, pos_count: ", res[(subj, sess)].shape, np.sum(res[(subj, sess)][:, 1]), np.sum(res[(subj, sess)][:, 1]>=0.5))            
            
    mfileu.write_file('all_proba', 'all_proba_bite_lab_DeepConvLSTM.pkl', res)
    print("Done Lab bite Testing")
    

In [ ]:
if train_test=='test_free':
    print("Bite Model Testing Free .....")
    ds = mfileu.read_file('data', 'free_data_steven_right_smoothed.pkl')    
    ds = normalize_data(ds)
    fs = mfileu.read_file('features', 'free_features_steven_right.pkl')
    
    res = {}    
    for subj in range(len(ds)):        
        for sess in range(len(ds[subj])):
            
            ix = fs[subj][sess][:, 0]
            test_indices = np.zeros((len(ix), 4)).astype(int)
            test_indices[:, 0] = subj
            test_indices[:, 1] = sess
            test_indices[:, 2] = ix            
            print("Subj, sess, indices shape: ", subj, sess, ix.shape, test_indices.shape)
                
            lab_subj = subj+2 if subj<5 else 100            
            model_path_src = model_folder_src+"/subj_"+str(lab_subj)            
            pred = train_test_model(ds, mu=mu, sigma=sigma, train_indices=[], test_indices=test_indices, model_path_src=model_path_src)            
            print("Prediction shape: ", pred.shape, np.sum(pred), np.sum(pred>=0.5))            
            assert len(pred)==len(test_indices)
            print(pred[:20])
            
            ixpred = np.zeros((len(pred), 2))
            ixpred[:, 0] = ix
            ixpred[:, 1] = pred            
            res[(subj, sess)] = ixpred
            print("Result shape, total_prob, pos_count: ", res[(subj, sess)].shape, np.sum(res[(subj, sess)][:, 1]), np.sum(res[(subj, sess)][:, 1]>=0.5))            
            
    mfileu.write_file('all_proba', 'all_proba_bite_free_DeepConvLSTM.pkl', res)
    print("Done Free Bite Testing")
    